## PDF Generation Workflow

Notebook for generating PDFs from resume-job matches with PII integration.

### Configuration:
- Set `SELECTION_CHOICE` for file selection (1-4)
- Configure `TREATMENT_TYPES` to process
- API settings loaded from `.env` file
- Results saved to data folder

### Workflow:
1. Load configuration and data
2. Select files to process
3. Generate PII data for each file/treatment
4. Test with single file (optional)
5. Process all files with production webhook
6. Save results and metadata to CSV

### Data Sources:
- Job matches: `ITC_resume_job_match_data.csv`
- Country mapping: `country_cluster_mapping.csv`
- PII clusters: `PII_country_cluster_mapping.csv`

### Imports

In [1]:
# Required imports
import requests
import json
import pandas as pd
import os
from datetime import datetime
import time  
import random
from dotenv import load_dotenv

load_dotenv()

True

### Configuration

In [34]:
# Configuration Settings
# =====================

# File Processing Configuration
SELECTION_CHOICE = "1"  # Options: "1" (all files), "2" (by number), "3" (by filename), "4" (first N files)
FILE_NUMBERS = [2]  # For option 2 - specific file numbers (1-based indexing)
FILENAMES = ["ITC resume 01.pdf"]  # For option 3 - specific filenames
NUM_FILES_TO_PROCESS = 4  # For option 4 - number of files to process

# Treatment Configuration
TREATMENT_TYPES = ['control', 'Type_I', 'Type_II', 'Type_III']

# Reprocessing Configuration
REPROCESS_ALREADY_PROCESSED = False  # Set to True to reprocess already processed files

# API Configuration - Load from .env file only
TEST_WEBHOOK_URL = os.getenv('TEST_WEBHOOK_URL')
PRODUCTION_WEBHOOK_URL = os.getenv('PRODUCTION_WEBHOOK_URL')
WEBHOOK_USERNAME = os.getenv('WEBHOOK_USERNAME')
WEBHOOK_PASSWORD = os.getenv('WEBHOOK_PASSWORD')

# Validate that all required environment variables are set
required_env_vars = {
    'TEST_WEBHOOK_URL': TEST_WEBHOOK_URL,
    'PRODUCTION_WEBHOOK_URL': PRODUCTION_WEBHOOK_URL,
    'WEBHOOK_USERNAME': WEBHOOK_USERNAME,
    'WEBHOOK_PASSWORD': WEBHOOK_PASSWORD
}

missing_vars = [var for var, value in required_env_vars.items() if not value]
if missing_vars:
    raise ValueError(f"Missing required environment variables: {missing_vars}")

AUTHORIZATION = (WEBHOOK_USERNAME, WEBHOOK_PASSWORD)
# Output Configuration
OUTPUT_CSV = "data/application_results.csv"

# Template Configuration
TEMPLATE_ASSIGNMENTS = {
    'control': {'template_id': '72b77b23d48f366e', 'markdown_template': 1},
    'Type_I': {'template_id': '73677b23d4896786', 'markdown_template': 2},
    'Type_II': {'template_id': '2ee77b23de5fc78e', 'markdown_template': 2},
    'Type_III': {'template_id': 'ca277b23d48328b0', 'markdown_template': 1}
}

print("Configuration Loaded:")
print(f"- Selection Choice: {SELECTION_CHOICE}")
print(f"- Reprocess Already Processed: {REPROCESS_ALREADY_PROCESSED}")
print(f"- Treatment Types: {TREATMENT_TYPES}")
print(f"- Output CSV: {OUTPUT_CSV}")
print(f"- Templates Configured: {len(TEMPLATE_ASSIGNMENTS)}")
print(f"- API Configuration: ✅ Loaded from .env file")

# Ensure data directory exists
os.makedirs("data", exist_ok=True)
print(f"✅ Data directory ensured: data/")

Configuration Loaded:
- Selection Choice: 1
- Reprocess Already Processed: False
- Treatment Types: ['control', 'Type_I', 'Type_II', 'Type_III']
- Output CSV: data/application_results.csv
- Templates Configured: 4
- API Configuration: ✅ Loaded from .env file
✅ Data directory ensured: data/


### Data Loading

In [35]:
# Load Job Matches Data
# ====================

# Load the job matches data
file_path = "ITC_resume_job_match_data.csv"
job_matches_df = pd.read_csv(file_path)

# Rename columns to match our endpoint requirements
job_matches_df.rename(columns={'tile': 'job_title'}, inplace=True)

print(f"✅ Loaded {len(job_matches_df)} job matches")
print(f"📋 Columns: {list(job_matches_df.columns)}")

# Display first few rows
print(f"\n📊 Sample data:")
job_matches_df.head()

✅ Loaded 67 job matches
📋 Columns: ['_id', 'job_posting_id', 'title', 'location', 'job_description', 'file_id', 'key_metrics.basics.likely_home_country', 'match_score', 'job_link']

📊 Sample data:


,_id,job_posting_id,title,location,job_description,file_id,key_metrics.basics.likely_home_country,match_score,job_link
0,68c8b9e9c71e0eac015015bb,68c70ea97ebc786e00455234,Product Designer,"Toronto, ON",# Product Designer\n\n### **About you and the ...,ITC resume 01.pdf,Lebanon,95,https://job-boards.greenhouse.io/partnerstack/...
1,68c8c73ee4085c93830e4b87,68c712787ebc786e00455350,Senior Program Manager,Remote,# Senior Program Manager\n\n**The Role:**\n\nR...,ITC resume 03.pdf,Bangladesh,95,https://job-boards.greenhouse.io/jobber/jobs/8...
2,68c8c760e4085c93830e4b8b,68c712787ebc786e00455355,Senior Product Manager,Remote,# Senior Product Manager\n\nWe are seeking a h...,ITC resume 02.pdf,Iran,88,https://job-boards.greenhouse.io/levelaccess/j...
3,68c8c94de4085c93830e4bbf,68c7127a7ebc786e00455392,Security Devops- Cryptography & Blockchain Pen...,Remote,# Security Devops- Cryptography & Blockchain P...,ITC resume 18.pdf,Eritrea,80,https://moduscreate.com/careers/7245851003?gh_...
4,68c8c9f2e4085c93830e4bd4,68c7127b7ebc786e004553a7,"Infrastructure Engineer, Conversational AI",Remote,"# Infrastructure Engineer, Conversational AI\n...",ITC resume 18.pdf,Eritrea,85,https://job-boards.greenhouse.io/glia/jobs/564...


### File Selection Logic

In [38]:
# File Selection and Display
# =========================

# Get all files to process (not unique)
all_files_df = job_matches_df.copy()
print(f"📁 Total files to process: {len(all_files_df)}")

# Load existing results to filter already processed files
existing_results = []
if os.path.exists(OUTPUT_CSV):
    existing_results = pd.read_csv(OUTPUT_CSV).to_dict('records')
    print(f"�� Loaded {len(existing_results)} existing results from {OUTPUT_CSV}")
else:
    print(f"📋 No existing results found.")

# Get unique job match IDs that have already been processed
processed_job_ids = set()
if existing_results and not REPROCESS_ALREADY_PROCESSED:
    for record in existing_results:
        job_id = record.get('id', '')  # This is the _id field from job_matches_df
        if job_id:
            processed_job_ids.add(job_id)
    
    print(f"�� Found {len(processed_job_ids)} already processed job matches")

# Filter out already processed job matches if not reprocessing
if not REPROCESS_ALREADY_PROCESSED and processed_job_ids:
    print(f"⏭️  Skipping {len(processed_job_ids)} already processed job matches")
    all_files_df = all_files_df[~all_files_df['_id'].isin(processed_job_ids)]
    print(f"📊 Remaining job matches to process: {len(all_files_df)}")

# Display first 10 files with their details
print(f"\n📋 Sample files to process (first 10):")
print("-" * 60)
for idx, row in all_files_df.head(10).iterrows():
    print(f"{idx+1:2d}. {row['file_id']}")
    if 'key_metrics.basics.likely_home_country' in row and pd.notna(row['key_metrics.basics.likely_home_country']):
        print(f"    Country: {row['key_metrics.basics.likely_home_country']}")
    if 'location' in row and pd.notna(row['location']):
        print(f"    Location: {row['location']}")
    print(f"    Job Title: {row.get('job_title', 'N/A')}")
    print()

if len(all_files_df) > 10:
    print(f"... and {len(all_files_df) - 10} more files")

# File selection logic
print(f"🎯 File Selection Options:")
print("1. Process all files")
print("2. Process specific files by number")
print("3. Process specific files by filename") 
print("4. Process first N files")
print(f"\nCurrent selection: Option {SELECTION_CHOICE}")
print(f"Reprocess already processed: {REPROCESS_ALREADY_PROCESSED}")

# Apply file selection logic
if SELECTION_CHOICE == "1":
    # Process all files
    files_to_process = all_files_df
    print(f"\n✅ Processing ALL {len(files_to_process)} job matches")
    
elif SELECTION_CHOICE == "2":
    # Process specific files by number
    print(f"\n✅ Processing specific files by number: {FILE_NUMBERS}")
    try:
        valid_indices = [i - 1 for i in FILE_NUMBERS if 1 <= i <= len(all_files_df)]
        if valid_indices:
            files_to_process = all_files_df.iloc[valid_indices]
            print(f"Selected {len(files_to_process)} job matches:")
            for idx, row in files_to_process.iterrows():
                print(f"  - {row['file_id']} (Job: {row.get('job_posting_id', 'N/A')})")
        else:
            print("⚠️  No valid file numbers provided. Processing first file.")
            files_to_process = all_files_df.head(1)
    except Exception as e:
        print(f"❌ Error: {e}. Processing first file.")
        files_to_process = all_files_df.head(1)
        
elif SELECTION_CHOICE == "3":
    # Process specific files by filename
    print(f"\n✅ Processing specific files by filename: {FILENAMES}")
    try:
        files_to_process = all_files_df[all_files_df['file_id'].isin(FILENAMES)]
        if len(files_to_process) > 0:
            print(f"Selected {len(files_to_process)} job matches:")
            for idx, row in files_to_process.iterrows():
                print(f"  - {row['file_id']} (Job: {row.get('job_posting_id', 'N/A')})")
        else:
            print("⚠️  No matching filenames found. Processing first file.")
            files_to_process = all_files_df.head(1)
    except Exception as e:
        print(f"❌ Error: {e}. Processing first file.")
        files_to_process = all_files_df.head(1)
        
elif SELECTION_CHOICE == "4":
    # Process first N files
    files_to_process = all_files_df.head(NUM_FILES_TO_PROCESS)
    print(f"\n✅ Processing first {NUM_FILES_TO_PROCESS} job matches:")
    for idx, row in files_to_process.iterrows():
        print(f"  - {row['file_id']} (Job: {row.get('job_posting_id', 'N/A')})")
        
else:
    # Default to first file if invalid input
    print("⚠️  Invalid choice. Processing first file.")
    files_to_process = all_files_df.head(1)

print(f"\n🎯 Final selection: {len(files_to_process)} job matches to process")

# Calculate total operations
total_operations = len(files_to_process) * len(TREATMENT_TYPES)
print(f"🔄 Total operations to process: {total_operations}")

📁 Total files to process: 67
�� Loaded 68 existing results from data/application_results.csv
�� Found 17 already processed job matches
⏭️  Skipping 17 already processed job matches
📊 Remaining job matches to process: 50

📋 Sample files to process (first 10):
------------------------------------------------------------
18. ITC resume 18.pdf
    Country: Eritrea
    Location: Remote
    Job Title: N/A

19. ITC resume 18.pdf
    Country: Eritrea
    Location: Remote
    Job Title: N/A

20. ITC resume 18.pdf
    Country: Eritrea
    Location: Remote
    Job Title: N/A

21. ITC resume 01.pdf
    Country: Lebanon
    Location: Remote
    Job Title: N/A

22. ITC resume 01.pdf
    Country: Lebanon
    Location: Remote
    Job Title: N/A

23. ITC resume 18.pdf
    Country: Eritrea
    Location: Vancouver, BC
    Job Title: N/A

24. ITC resume 18.pdf
    Country: Eritrea
    Location: Remote
    Job Title: N/A

25. ITC resume 18.pdf
    Country: Eritrea
    Location: Remote
    Job Title: N/A

2

### PII Data Loading and Functions

In [39]:
# PII Data Loading and Functions
# =============================

# Load the country to geographic cluster mapping
country_cluster_df = pd.read_csv("country_cluster_mapping.csv")
print(f"✅ Loaded country mapping: {len(country_cluster_df)} countries mapped to {country_cluster_df['Geographic_Cluster'].nunique()} clusters")

# Load the PII clusters data
pii_clusters_df = pd.read_csv("PII_country_cluster_mapping.csv")
print(f"✅ Loaded PII clusters: {len(pii_clusters_df)} cluster-treatment combinations")

# Create lookup dictionaries for efficient access
country_to_cluster = dict(zip(country_cluster_df['Country'], country_cluster_df['Geographic_Cluster']))

# Create a nested dictionary for PII lookup: {cluster: {treatment: pii_data}}
pii_lookup = {}
for _, row in pii_clusters_df.iterrows():
    cluster = row['Geographic_Cluster']
    treatment = row['Treatment Type']
    
    if cluster not in pii_lookup:
        pii_lookup[cluster] = {}
    
    # Parse the name pools into lists
    male_names = [name.strip() for name in row['Male_First_Name_Pool'].split(',')]
    female_names = [name.strip() for name in row['Female_First_Name_Pool'].split(',')]
    
    pii_lookup[cluster][treatment] = {
        'last_name': row['Last_Name'],
        'email': row['Assigned_Email'],
        'phone': row['Assigned_Phone_Number'],
        'male_names': male_names,
        'female_names': female_names
    }

print(f"\n📋 PII lookup structure created:")
for cluster in pii_lookup:
    print(f"  {cluster}: {list(pii_lookup[cluster].keys())}")

# Function to get PII data for a country and treatment type
def get_pii_data(country, treatment_type):
    """
    Get PII data for a given country and treatment type.
    
    Args:
        country (str): Country name
        treatment_type (str): Treatment type (control, Type_I, Type_II, Type_III)
    
    Returns:
        dict: PII data with keys: last_name, email, phone, male_names, female_names
        None: If country or treatment not found
    """
    # Map country to geographic cluster
    if country not in country_to_cluster:
        print(f"⚠️  Warning: Country '{country}' not found in mapping")
        return None
    
    cluster = country_to_cluster[country]
    
    # Get PII data for cluster and treatment
    if cluster not in pii_lookup or treatment_type not in pii_lookup[cluster]:
        print(f"⚠️  Warning: No PII data found for cluster '{cluster}' and treatment '{treatment_type}'")
        return None
    
    return pii_lookup[cluster][treatment_type]

# Function to generate a random name and PII
def generate_pii_for_file(country, treatment_type):
    """
    Generate PII data for a file based on country and treatment type.
    
    Args:
        country (str): Country name
        treatment_type (str): Treatment type
    
    Returns:
        dict: Complete PII data with full_name, email, phone
        None: If PII data cannot be generated
    """
    pii_data = get_pii_data(country, treatment_type)
    if not pii_data:
        return None
    
    # Randomly select gender (50/50 chance)
    is_male = random.choice([True, False])
    
    # Select random name from appropriate pool
    if is_male:
        first_name = random.choice(pii_data['male_names'])
    else:
        first_name = random.choice(pii_data['female_names'])
    
    # Construct full name
    full_name = f"{first_name} {pii_data['last_name']}"
    
    return {
        'full_name': full_name,
        'email': pii_data['email'],
        'phone': pii_data['phone'],
        'first_name': first_name,
        'last_name': pii_data['last_name'],
        'gender': 'Male' if is_male else 'Female'
    }

# Test the functions with a sample
print(f"\n🧪 Testing PII generation:")
test_country = "India"
test_treatment = "Type_II"
test_pii = generate_pii_for_file(test_country, test_treatment)
if test_pii:
    print(f"  Country: {test_country}, Treatment: {test_treatment}")
    print(f"  Generated: {test_pii['full_name']} ({test_pii['gender']})")
    print(f"  Email: {test_pii['email']}")
    print(f"  Phone: {test_pii['phone']}")
else:
    print(f"  ❌ Failed to generate PII for {test_country} - {test_treatment}")

✅ Loaded country mapping: 63 countries mapped to 7 clusters
✅ Loaded PII clusters: 28 cluster-treatment combinations

📋 PII lookup structure created:
  South Asia: ['control', 'Type_I', 'Type_II', 'Type_III']
  Sub-Saharan Africa: ['control', 'Type_I', 'Type_II', 'Type_III']
  Middle East & North Africa: ['control', 'Type_I', 'Type_II', 'Type_III']
  East & Southeast Asia: ['control', 'Type_I', 'Type_II', 'Type_III']
  Eastern Europe: ['control', 'Type_I', 'Type_II', 'Type_III']
  Latin America: ['control', 'Type_I', 'Type_II', 'Type_III']
  Anglosphere: ['control', 'Type_I', 'Type_II', 'Type_III']

🧪 Testing PII generation:
  Country: India, Treatment: Type_II
  Generated: Rohan Kumar (Male)
  Email: kumarx0349@gmail.com
  Phone: +1 (437) 524-5048


### Apply PII Data to Selected Files

In [40]:
# Apply PII Data to Selected Files
# ===============================

# Apply PII selection to the files we want to process
print("🔄 Generating PII data for selected files...")
print("-" * 60)

# Add PII data to our files_to_process dataframe
files_with_pii = []

for idx, row in files_to_process.iterrows():
    file_id = row['file_id']
    country = row.get('key_metrics.basics.likely_home_country', 'Unknown')
    
    print(f"\n📄 File: {file_id}")
    print(f"   Country: {country}")
    
    file_pii_data = {}
    
    # Generate PII for each treatment type
    for treatment_type in TREATMENT_TYPES:
        if pd.isna(country) or country == 'Unknown':
            print(f"   ⚠️  Warning: No country data for {treatment_type}, using default PII")
            # Use a default (you can customize this)
            file_pii_data[treatment_type] = {
                'full_name': 'Test User',
                'email': 'test@example.com',
                'phone': '123-456-7890',
                'first_name': 'Test',
                'last_name': 'User',
                'gender': 'Unknown'
            }
        else:
            pii = generate_pii_for_file(country, treatment_type)
            if pii:
                file_pii_data[treatment_type] = pii
                print(f"   {treatment_type}: {pii['full_name']} ({pii['gender']})")
                print(f"     Email: {pii['email']}")
                print(f"     Phone: {pii['phone']}")
            else:
                print(f"   {treatment_type}: Failed to generate PII")
                # Fallback to default
                file_pii_data[treatment_type] = {
                    'full_name': 'Test User',
                    'email': 'test@example.com',
                    'phone': '123-456-7890',
                    'first_name': 'Test',
                    'last_name': 'User',
                    'gender': 'Unknown'
                }
    
    # Store the file data with PII
    file_data = {
        'file_row': row,
        'pii_data': file_pii_data
    }
    files_with_pii.append(file_data)

print(f"\n✅ PII generation completed for {len(files_with_pii)} files")
print(f"�� Total operations to process: {len(files_with_pii)} files × {len(TREATMENT_TYPES)} treatments = {len(files_with_pii) * len(TREATMENT_TYPES)} operations")

🔄 Generating PII data for selected files...
------------------------------------------------------------

📄 File: ITC resume 18.pdf
   Country: Eritrea
   control: Louis Adebayo (Male)
     Email: adebayo01x@gmail.com
     Phone: +1 (647)-948-8050
   Type_I: Michael Okoro (Male)
     Email: okoro0293x@gmail.com
     Phone: +1 (647)-558-4334
   Type_II: Mamadou Traoré (Male)
     Email: traore92tr@gmail.com
     Phone: +1 (437) 524-5048
   Type_III: Hugo Mensah (Male)
     Email: mensah87452@gmail.com
     Phone: +1 (437) 837-0433	

📄 File: ITC resume 18.pdf
   Country: Eritrea
   control: Samuel Adebayo (Male)
     Email: adebayo01x@gmail.com
     Phone: +1 (647)-948-8050
   Type_I: Akosua Okoro (Female)
     Email: okoro0293x@gmail.com
     Phone: +1 (647)-558-4334
   Type_II: Chioma Traoré (Female)
     Email: traore92tr@gmail.com
     Phone: +1 (437) 524-5048
   Type_III: Mariama Mensah (Female)
     Email: mensah87452@gmail.com
     Phone: +1 (437) 837-0433	

📄 File: ITC resume 18.

### Webhook Request Body Creation Function

In [41]:
# Request Body Creation Function
# =============================

# Enhanced webhook request function that includes PII data
def create_enhanced_request_body(file_row, treatment_type, pii_data):
    """
    Create an enhanced request body with PII data.
    
    Args:
        file_row: Row from the job matches dataframe
        treatment_type: Treatment type being processed
        pii_data: PII data for this treatment type
    
    Returns:
        dict: Enhanced request body
    """
    # Handle location logic: Convert "Remote" to "Toronto, ON"
    location = file_row.get('location', 'Toronto, ON')
    if pd.isna(location) or location == 'Remote':
        location = 'Toronto, ON'
    
    request_body = {
        'id': file_row['_id'],
        'file_id': file_row['file_id'],
        'treatment_type': treatment_type,
        'name': pii_data['full_name'],
        'email': pii_data['email'],
        'phone': pii_data['phone'],
        'location': location,
        'first_name': pii_data['first_name'],
        'last_name': pii_data['last_name'],
        'gender': pii_data['gender'],
        'country': file_row.get('key_metrics.basics.likely_home_country', 'Unknown'),
        'geographic_cluster': country_to_cluster.get(file_row.get('key_metrics.basics.likely_home_country', ''), 'Unknown'),
        # Add job-related fields
        'job_description': file_row.get('job_description', ''),
        'job_title': file_row.get('title', ''),
        'job_posting_id': file_row.get('job_posting_id', ''),
        'match_score': file_row.get('match_score', ''),
        'job_link': file_row.get('job_link', '')
    }
    
    return request_body

# Test the request body creation
print("🧪 Testing request body creation:")
print("-" * 60)

if files_with_pii:
    test_file = files_with_pii[0]
    test_treatment = TREATMENT_TYPES[0]
    test_pii = test_file['pii_data'][test_treatment]
    
    test_request = create_enhanced_request_body(
        test_file['file_row'], 
        test_treatment, 
        test_pii
    )
    
    print(f"�� Sample request body for {test_file['file_row']['file_id']} - {test_treatment}:")
    for key, value in test_request.items():
        if key == 'job_description':
            # Truncate job description for display
            display_value = value[:100] + "..." if len(str(value)) > 100 else value
            print(f"   {key}: {display_value}")
        else:
            print(f"   {key}: {value}")
else:
    print("❌ No files with PII data found. Please run the previous cell first.")

🧪 Testing request body creation:
------------------------------------------------------------
�� Sample request body for ITC resume 18.pdf - control:
   id: 68c8cf79e4085c93830e4c6d
   file_id: ITC resume 18.pdf
   treatment_type: control
   name: Louis Adebayo
   email: adebayo01x@gmail.com
   phone: +1 (647)-948-8050
   location: Toronto, ON
   first_name: Louis
   last_name: Adebayo
   gender: Male
   country: Eritrea
   geographic_cluster: Sub-Saharan Africa
   job_description: # Sr. AWS Developer\n\n### **Responsibilities:**\n\n*   Work closely with a Solution Architect to de...
   job_title: Sr. AWS Developer
   job_posting_id: 68c72dc07ebc786e00455a7c
   match_score: 75
   job_link: https://job-boards.greenhouse.io/neuraflash/jobs/5644988004?gh_src=my.greenhouse.search


### Single File Test (Test Webhook)

In [42]:
# Single File Test - Test Webhook
# ===============================

print("=== SINGLE FILE TEST ===")
print(f"Testing webhook: {TEST_WEBHOOK_URL}")

# Get first file for testing
if 'files_with_pii' in locals() and len(files_with_pii) > 0:
    test_file = files_with_pii[1]
    test_treatment = 'Type_II' # 'control', 'Type_I', 'Type_II', 'Type_III'
    
    # Extract data from the correct structure
    file_row = test_file['file_row']
    file_id = file_row['file_id']
    job_title = file_row.get('title', 'N/A')
    job_description = file_row.get('job_description', '')
    
    print(f"📄 Testing file: {file_id}")
    print(f"💼 Job Title: {job_title}")
    print(f"🧪 Treatment: {test_treatment}")
    
    # Get PII data
    treatment_pii = test_file['pii_data'][test_treatment]
    print(f"�� PII: {treatment_pii['full_name']} ({treatment_pii['email']})")
    
    # Create request body using the correct data structure
    request_body = create_enhanced_request_body(file_row, test_treatment, treatment_pii)
    
    # Add template configuration
    template_config = TEMPLATE_ASSIGNMENTS[test_treatment]
    request_body['template_id'] = template_config['template_id']
    request_body['markdown_template'] = template_config['markdown_template']
    
    print(f"�� Request body keys: {list(request_body.keys())}")
    print(f"🎨 Template ID: {request_body['template_id']}")
    print(f"📝 Markdown Template: {request_body['markdown_template']}")
    
    # Send request
    print("\n📤 Sending test request...")
    try:
        response = requests.post(
            TEST_WEBHOOK_URL,
            json=request_body,
            auth=AUTHORIZATION,
            timeout=120
        )
        
        print(f"📊 Response Status: {response.status_code}")
        
        if response.status_code == 200:
            response_data = response.json()
            print(f"✅ Response received: {response_data}")
            
            # Extract the actual values
            web_view_link = response_data.get('webViewLink', '')
            pdf_id = response_data.get('id', '')
            pdf_filename = response_data.get('name', '')
            pdf_mime_type = response_data.get('mimeType', '')
            status = response_data.get('status', '')
            
            print("\n🔍 Extracted Data:")
            print(f"  ✅ Web View Link: {web_view_link}")
            print(f"  ✅ PDF ID: {pdf_id}")
            print(f"  ✅ Filename: {pdf_filename}")
            print(f"  ✅ MIME Type: {pdf_mime_type}")
            print(f"  ✅ Status: {status}")
            
            # Verify we got actual values
            if web_view_link and web_view_link.startswith('https://drive.google.com'):
                print("\n🎉 SUCCESS:")
                print(f"   🔗 PDF Link: {web_view_link}")
            else:
                print(f"\n❌ ISSUE: Invalid webViewLink: {web_view_link}")
                
        else:
            print(f"❌ Error: {response.status_code}")
            print(f"Response text: {response.text}")
            
    except Exception as e:
        print(f"❌ Exception: {str(e)}")
        
else:
    print("❌ No files_with_pii data found. Run the previous cells first.")

=== SINGLE FILE TEST ===
Testing webhook: https://adhil-is-dummy.app.n8n.cloud/webhook-test/9eb0c4bc-f2a4-4f23-bb71-26422deedf55
📄 Testing file: ITC resume 18.pdf
💼 Job Title: Intermediate Site Reliability Engineer
🧪 Treatment: Type_II
�� PII: Chioma Traoré (traore92tr@gmail.com)
�� Request body keys: ['id', 'file_id', 'treatment_type', 'name', 'email', 'phone', 'location', 'first_name', 'last_name', 'gender', 'country', 'geographic_cluster', 'job_description', 'job_title', 'job_posting_id', 'match_score', 'job_link', 'template_id', 'markdown_template']
🎨 Template ID: 2ee77b23de5fc78e
📝 Markdown Template: 2

📤 Sending test request...
📊 Response Status: 200
✅ Response received: {'webViewLink': 'https://drive.google.com/file/d/19g6HXtgDbeIOk_00YO2yKtSxZcx3mt7j/view?usp=drivesdk', 'id': '19g6HXtgDbeIOk_00YO2yKtSxZcx3mt7j', 'name': 'Chioma_Traoré_Resume', 'mimeType': 'application/pdf', 'status': 'success'}

🔍 Extracted Data:
  ✅ Web View Link: https://drive.google.com/file/d/19g6HXtgDbeIOk

### Production File Processing

In [43]:
# Production File Processing
# ==========================

print("=== PRODUCTION PROCESSING ===")
print(f"Production webhook: {PRODUCTION_WEBHOOK_URL}")
print(f"Processing {len(files_with_pii)} files with {len(TREATMENT_TYPES)} treatments each")
print(f"Total operations: {len(files_with_pii) * len(TREATMENT_TYPES)}")

# Load existing results if available
existing_results = []
if os.path.exists(OUTPUT_CSV):
    existing_results = pd.read_csv(OUTPUT_CSV).to_dict('records')
    print(f"�� Loaded {len(existing_results)} existing results from {OUTPUT_CSV}")
else:
    print(f"�� No existing results found. Will create new {OUTPUT_CSV}")

# Initialize results list
new_results = []
total_operations = len(files_with_pii) * len(TREATMENT_TYPES)
current_operation = 0

print(f"\n�� Starting processing...")
print("=" * 60)

for file_idx, file_data in enumerate(files_with_pii, 1):
    # Extract file_row and pii_data from the correct structure
    file_row = file_data['file_row']
    file_id = file_row['file_id']
    pii_data = file_data['pii_data']
    
    print(f"\n📄 Processing file {file_idx}/{len(files_with_pii)}: {file_id}")
    print("-" * 40)
    
    for treatment_idx, treatment_type in enumerate(TREATMENT_TYPES, 1):
        current_operation += 1
        treatment_pii = pii_data[treatment_type]
        
        print(f"  �� Treatment {treatment_idx}/{len(TREATMENT_TYPES)}: {treatment_type}")
        print(f"     👤 Using PII: {treatment_pii['full_name']} ({treatment_pii['email']})")
        
        # Get template configuration for this treatment type
        template_config = TEMPLATE_ASSIGNMENTS[treatment_type]
        print(f"     🎨 Template ID: {template_config['template_id']}")
        print(f"     �� Markdown Template: {template_config['markdown_template']}")
        
        # Create enhanced request body
        request_body = create_enhanced_request_body(file_row, treatment_type, treatment_pii)
        request_body['template_id'] = template_config['template_id']
        request_body['markdown_template'] = template_config['markdown_template']
        
        print(f"     📤 Sending request...")
        
        try:
            # Send request to n8n webhook
            response = requests.post(
                PRODUCTION_WEBHOOK_URL,
                json=request_body,
                auth=AUTHORIZATION,
                timeout=120
            )
            
            if response.status_code == 200:
                response_data = response.json()
                
                # Extract response data
                web_view_link = response_data.get('webViewLink', '')
                pdf_id = response_data.get('id', '')
                pdf_filename = response_data.get('name', '')
                pdf_mime_type = response_data.get('mimeType', '')
                status = response_data.get('status', '')
                
                # Create result record
                result_record = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'id': request_body['id'],
                    'file_id': file_id,
                    'treatment_type': treatment_type,
                    'template_id': template_config['template_id'],
                    'markdown_template': template_config['markdown_template'],
                    'pdf_web_view_link': web_view_link,
                    'pdf_file_id': pdf_id,
                    'pdf_filename': pdf_filename,
                    'pdf_mime_type': pdf_mime_type,
                    'pdf_status': status,
                    # PII data
                    'name': treatment_pii['full_name'],
                    'email': treatment_pii['email'],
                    'phone': treatment_pii['phone'],
                    'first_name': treatment_pii['first_name'],
                    'last_name': treatment_pii['last_name'],
                    'gender': treatment_pii['gender'],
                    'country': file_row.get('key_metrics.basics.likely_home_country', ''),
                    'geographic_cluster': country_to_cluster.get(file_row.get('key_metrics.basics.likely_home_country', ''), ''),
                    'location': request_body['location'],
                    # Job data
                    'job_posting_id': file_row.get('job_posting_id', ''),
                    'job_title': file_row.get('title', ''),
                    'job_description': file_row.get('job_description', '')[:200] + '...' if len(str(file_row.get('job_description', ''))) > 200 else file_row.get('job_description', ''),
                    'match_score': file_row.get('match_score', ''),
                    'job_link': file_row.get('job_link', '')
                }
                
                new_results.append(result_record)
                
                print(f"     ✅ Success: PDF generated")
                print(f"        🔗 Web View Link: {web_view_link}")
                print(f"        �� Filename: {pdf_filename}")
                
            else:
                print(f"     ❌ Error: {response.status_code}")
                print(f"        Response: {response.text}")
                
                # Create error record
                error_record = {
                    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'id': request_body['id'],
                    'file_id': file_id,
                    'treatment_type': treatment_type,
                    'template_id': template_config['template_id'],
                    'markdown_template': template_config['markdown_template'],
                    'pdf_web_view_link': '',
                    'pdf_file_id': '',
                    'pdf_filename': '',
                    'pdf_mime_type': '',
                    'pdf_status': f'Error: {response.status_code}',
                    # PII data
                    'name': treatment_pii['full_name'],
                    'email': treatment_pii['email'],
                    'phone': treatment_pii['phone'],
                    'first_name': treatment_pii['first_name'],
                    'last_name': treatment_pii['last_name'],
                    'gender': treatment_pii['gender'],
                    'country': file_row.get('key_metrics.basics.likely_home_country', ''),
                    'geographic_cluster': country_to_cluster.get(file_row.get('key_metrics.basics.likely_home_country', ''), ''),
                    'location': request_body['location'],
                    # Job data
                    'job_posting_id': file_row.get('job_posting_id', ''),
                    'job_title': file_row.get('title', ''),
                    'job_description': file_row.get('job_description', '')[:200] + '...' if len(str(file_row.get('job_description', ''))) > 200 else file_row.get('job_description', ''),
                    'match_score': file_row.get('match_score', ''),
                    'job_link': file_row.get('job_link', '')
                }
                
                new_results.append(error_record)
                
        except Exception as e:
            print(f"     ❌ Exception: {str(e)}")
            
            # Create exception record
            exception_record = {
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'id': request_body.get('id', ''),
                'file_id': file_id,
                'treatment_type': treatment_type,
                'template_id': template_config['template_id'],
                'markdown_template': template_config['markdown_template'],
                'pdf_web_view_link': '',
                'pdf_file_id': '',
                'pdf_filename': '',
                'pdf_mime_type': '',
                'pdf_status': f'Exception: {str(e)}',
                # PII data
                'name': treatment_pii['full_name'],
                'email': treatment_pii['email'],
                'phone': treatment_pii['phone'],
                'first_name': treatment_pii['first_name'],
                'last_name': treatment_pii['last_name'],
                'gender': treatment_pii['gender'],
                'country': file_row.get('key_metrics.basics.likely_home_country', ''),
                'geographic_cluster': country_to_cluster.get(file_row.get('key_metrics.basics.likely_home_country', ''), ''),
                'location': request_body.get('location', ''),
                # Job data
                'job_posting_id': file_row.get('job_posting_id', ''),
                'job_title': file_row.get('title', ''),
                'job_description': file_row.get('job_description', '')[:200] + '...' if len(str(file_row.get('job_description', ''))) > 200 else file_row.get('job_description', ''),
                'match_score': file_row.get('match_score', ''),
                'job_link': file_row.get('job_link', '')
            }
            
            new_results.append(exception_record)
        
        # Progress tracking
        progress_percent = (current_operation / total_operations) * 100
        print(f"     📊 Progress: {current_operation}/{total_operations} ({progress_percent:.1f}%)")
        
        # Rate limiting - wait between requests
        if current_operation < total_operations:
            print(f"     ⏳ Waiting 2 seconds before next request...")
            time.sleep(2)

print(f"\n🎉 Processing completed!")
print(f"📊 Generated {len(new_results)} records")
print(f"✅ Success rate: {len([r for r in new_results if 'success' in str(r.get('pdf_status', '')).lower()])}/{len(new_results)} successful")

=== PRODUCTION PROCESSING ===
Production webhook: https://adhil-is-dummy.app.n8n.cloud/webhook/9eb0c4bc-f2a4-4f23-bb71-26422deedf55
Processing 50 files with 4 treatments each
Total operations: 200
�� Loaded 68 existing results from data/application_results.csv

�� Starting processing...

📄 Processing file 1/50: ITC resume 18.pdf
----------------------------------------
  �� Treatment 1/4: control
     👤 Using PII: Louis Adebayo (adebayo01x@gmail.com)
     🎨 Template ID: 72b77b23d48f366e
     �� Markdown Template: 1
     📤 Sending request...
     ✅ Success: PDF generated
        🔗 Web View Link: https://drive.google.com/file/d/1Zc5RglFZzOsHFcnmzRRXyFWqftbwIh8_/view?usp=drivesdk
        �� Filename: Louis_Resume_Sr. AWS Developer
     📊 Progress: 1/200 (0.5%)
     ⏳ Waiting 2 seconds before next request...
  �� Treatment 2/4: Type_I
     👤 Using PII: Michael Okoro (okoro0293x@gmail.com)
     🎨 Template ID: 73677b23d4896786
     �� Markdown Template: 2
     📤 Sending request...
     ✅ Suc

### Save Results and Summary

In [13]:
# Save Results and Summary
# =======================

# Combine existing and new results
all_results = existing_results + new_results
results_df = pd.DataFrame(all_results)

# Save to CSV
results_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
print(f"✅ Results saved to {OUTPUT_CSV}")
print(f"📊 Total records: {len(results_df)}")
print(f"🆕 New records added: {len(new_results)}")

# Display summary statistics
success_count = len([r for r in new_results if 'success' in str(r.get('pdf_status', '')).lower()])
error_count = len([r for r in new_results if 'error' in str(r.get('pdf_status', '')).lower()])
exception_count = len([r for r in new_results if 'exception' in str(r.get('pdf_status', '')).lower()])

print(f"\n�� Processing Summary:")
print(f"  ✅ Successful: {success_count}")
print(f"  ❌ Errors: {error_count}")
print(f"  ⚠️  Exceptions: {exception_count}")
print(f"  📊 Success Rate: {(success_count/len(new_results)*100):.1f}%" if new_results else "N/A")

# Display treatment type breakdown
if new_results:
    print(f"\n🔬 Treatment Type Breakdown:")
    treatment_counts = {}
    for record in new_results:
        treatment = record.get('treatment_type', 'Unknown')
        status = record.get('pdf_status', 'Unknown')
        if treatment not in treatment_counts:
            treatment_counts[treatment] = {'success': 0, 'error': 0, 'exception': 0}
        
        if 'success' in str(status).lower():
            treatment_counts[treatment]['success'] += 1
        elif 'error' in str(status).lower():
            treatment_counts[treatment]['error'] += 1
        else:
            treatment_counts[treatment]['exception'] += 1
    
    for treatment, counts in treatment_counts.items():
        total = counts['success'] + counts['error'] + counts['exception']
        print(f"  {treatment}: {counts['success']}/{total} successful ({counts['success']/total*100:.1f}%)")

# Show sample of results
print(f"\n📋 Sample Results (first 5):")
display_columns = ['timestamp', 'file_id', 'treatment_type', 'pdf_status', 'name', 'email', 'pdf_web_view_link']
available_columns = [col for col in display_columns if col in results_df.columns]
if available_columns:
    sample_df = results_df[available_columns].head()
    for _, row in sample_df.iterrows():
        print(f"  �� {row['file_id']} - {row['treatment_type']}: {row['pdf_status']}")
        print(f"     👤 {row.get('name', 'N/A')} ({row.get('email', 'N/A')})")
        if row.get('pdf_web_view_link'):
            print(f"     🔗 {row['pdf_web_view_link']}")
        print()

print(f"\n�� Processing workflow completed!")
print(f"�� Results saved to: {OUTPUT_CSV}")
print(f"�� Total operations processed: {len(new_results)}")

✅ Results saved to data/application_results.csv
📊 Total records: 152
🆕 New records added: 136

�� Processing Summary:
  ✅ Successful: 54
  ❌ Errors: 2
  ⚠️  Exceptions: 80
  📊 Success Rate: 39.7%

🔬 Treatment Type Breakdown:
  control: 14/34 successful (41.2%)
  Type_I: 14/34 successful (41.2%)
  Type_II: 13/34 successful (38.2%)
  Type_III: 13/34 successful (38.2%)

📋 Sample Results (first 5):
  �� ITC resume 01.pdf - control: success
     👤 Mariam Hassan (hassan947h@gmail.com)
     🔗 https://drive.google.com/file/d/1YfBaFJGkNektLw9mX5tksCFY_2LU2iqL/view?usp=drivesdk

  �� ITC resume 01.pdf - Type_I: success
     👤 Bilal Rahman (rahman.rah2299@gmail.com)
     🔗 https://drive.google.com/file/d/1maWovLvVtHs4Dd3gZRwTeSeUsBzgOLVl/view?usp=drivesdk

  �� ITC resume 01.pdf - Type_II: success
     👤 Leila Karimi (karimi336ka@gmail.com)
     🔗 https://drive.google.com/file/d/1ejCeLMRwfGUjBgNHxSuAr5pJGhcmTJn8/view?usp=drivesdk

  �� ITC resume 01.pdf - Type_III: success
     👤 Leila Mohamed (m